# Microsoft AutoGen: Building Conversational Agent Teams

**Objective:** This notebook introduces Microsoft AutoGen, a powerful framework for creating multi-agent systems that solve tasks through conversation. You will learn how to set up a team of agents that can write code, execute it, and collaborate to achieve a user's goal.

**Target Audience:** Software engineers attending the AI-Driven Software Engineering Program.

**Core Philosophy:** AutoGen's core idea is that many complex tasks can be solved by a team of specialized agents conversing with each other. It provides a flexible and powerful platform for defining these agents and orchestrating their interactions.

## 1. Setup

We will install `pyautogen` and configure our LLM provider. AutoGen is optimized for the OpenAI API format, so we will use that for this lab.

In [1]:
import os
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    print("ERROR: OPENAI_API_KEY not found. Please check your .env file.")

## 2. Foundational Two-Agent Chat: The Coder and the Proxy

Our first example is the most common AutoGen pattern: a two-agent system for code generation.

-   **`AssistantAgent`**: This is the primary AI agent. It's an LLM-powered agent that can write text, and importantly, code.
-   **`UserProxyAgent`**: This agent acts as a proxy for the human user. Its key capability is that it can **execute code** sent by the `AssistantAgent`. This creates a powerful loop: the user gives a task, the assistant writes code, the proxy runs the code and reports the result (success or error), and the assistant can then debug or continue based on that feedback.

In [2]:
import autogen 

# AutoGen requires a 'config_list' to define the LLM to use.
config_list = [
    {
        'model': 'gemini-2.5-pro',
        'api_key': os.getenv("GOOGLE_API_KEY"),
    }
]

# The LLM configuration for the agents
llm_config = {"config_list": config_list}

# 1. The Coder Agent
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)

# 2. The User Proxy Agent (with code execution)
user_proxy = autogen.UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER", # The conversation runs automatically
    max_consecutive_auto_reply=8,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "autogen_work_dir", # Directory to save and execute code
        "use_docker": False,  # Set to True for safer execution if you have Docker
    },
)

# 3. Initiate the conversation
print("--- Initiating Two-Agent Coding Chat ---")
user_proxy.initiate_chat(
    coder,
    message="What is the current date? Write the Python code to find out and print it.",
)

[autogen.oai.client: 10-31 10:29:46] {351} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
--- Initiating Two-Agent Coding Chat ---
UserProxy (to Coder):

What is the current date? Write the Python code to find out and print it.

--------------------------------------------------------------------------------


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: AIzaSyBZ***************************_C7Q. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

## 3. Advanced Agentic Capability: A Three-Agent Team with a Reviewer

Now, let's build a more realistic software development workflow by adding a third agent: a `CodeReviewer`. This demonstrates AutoGen's `GroupChat` feature, where multiple agents can collaborate in a more complex conversation.

The flow will be:
1.  The `UserProxy` gives a task.
2.  The `Coder` writes the code.
3.  The `CodeReviewer` critiques the code.
4.  The `Coder` revises the code based on the feedback.
5.  The `UserProxy` executes the final, approved code.

In [3]:
# We can reuse the UserProxy and Coder from the previous example

# 1. Create the Code Reviewer Agent
reviewer = autogen.AssistantAgent(
    name="CodeReviewer",
    system_message="You are a senior software engineer. Your role is to review code for quality, correctness, and best practices. You must provide constructive feedback. If the code is perfect, you must reply with the word 'APPROVED'.",
    llm_config=llm_config,
)

# 2. Set up the Group Chat
groupchat = autogen.GroupChat(
    agents=[user_proxy, coder, reviewer],
    messages=[],
    max_round=12
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# 3. Initiate the group chat
print("\n--- Initiating Three-Agent Group Chat with Reviewer ---")
user_proxy.initiate_chat(
    manager,
    message="Write a Python function that takes a list of numbers and returns a new list containing only the even numbers. Then, call the function with an example and print the result.",
)


--- Initiating Three-Agent Group Chat with Reviewer ---
UserProxy (to chat_manager):

Write a Python function that takes a list of numbers and returns a new list containing only the even numbers. Then, call the function with an example and print the result.

--------------------------------------------------------------------------------



Next speaker: Coder

Coder (to chat_manager):

To accomplish this task, I'll write a Python function that filters out the even numbers from a list. We'll use list comprehension to make it concise and efficient. After defining the function, I'll call it with a sample list and print the result.

Below is the Python code:

```python
def filter_even_numbers(numbers):
    """Returns a list of even numbers from the provided list."""
    return [num for num in numbers if num % 2 == 0]

# Example usage
example_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
even_numbers = filter_even_numbers(example_list)
print("Even numbers:", even_numbers)
```

When you run this code block, it will print a list containing only the even numbers from the provided example list. TERMINATE

--------------------------------------------------------------------------------

Next speaker: CodeReviewer

CodeReviewer (to chat_manager):

The code you provided is well-written, clear, and makes efficient use of list comprehension

ChatResult(chat_id=None, chat_history=[{'content': 'Write a Python function that takes a list of numbers and returns a new list containing only the even numbers. Then, call the function with an example and print the result.', 'role': 'assistant', 'name': 'UserProxy'}, {'content': 'To accomplish this task, I\'ll write a Python function that filters out the even numbers from a list. We\'ll use list comprehension to make it concise and efficient. After defining the function, I\'ll call it with a sample list and print the result.\n\nBelow is the Python code:\n\n```python\ndef filter_even_numbers(numbers):\n    """Returns a list of even numbers from the provided list."""\n    return [num for num in numbers if num % 2 == 0]\n\n# Example usage\nexample_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\neven_numbers = filter_even_numbers(example_list)\nprint("Even numbers:", even_numbers)\n```\n\nWhen you run this code block, it will print a list containing only the even numbers from the provided example

## Lab Conclusion

This lab has introduced you to the core concepts of Microsoft AutoGen. You have built both a simple two-agent system and a more complex three-agent group chat, demonstrating how AutoGen facilitates collaborative problem-solving through conversation.

**Key Takeaways:**
- AutoGen excels at creating conversational agents that can collaborate.
- The `UserProxyAgent` is a powerful construct that can act on behalf of the user, including executing code.
- `GroupChat` allows you to orchestrate complex workflows with multiple specialized agents, mimicking a real-world development team.
- The system messages for each agent are crucial for defining their roles and behaviors within the conversation.